# YOLO Crater Detection Training Notebook
Complete end-to-end training pipeline in one notebook

## 1. Install Dependencies

In [7]:
import subprocess
import sys

# Install required packages
packages = ['ultralytics', 'opencv-python', 'torch', 'torchvision']
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', pkg, '-q'])
    
print("✓ All dependencies installed")

✓ All dependencies installed


## 2. Import Libraries

In [ ]:
import csv
import mathaa
import random
import shutil
from pathlib import Path
from typing import Dict, List, Tuple

import cv2
import torch
from ultralytics import YOLO
import numpy as np

print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  Device: {torch.cuda.get_device_name(0)}")

✓ PyTorch: 2.9.1+cpu
✓ GPU Available: False


## 3. Configuration

In [9]:
# ========== CONFIGURATION ==========
CONFIG = {
    'gt_csv': Path('../train-sb/train-gt.csv'),
    'images_root': Path('../train/train'),
    'dataset_out': Path('./dataset'),
    'runs_dir': Path('./runs'),
    'model': 'yolov8n',  # yolov8n, yolov8s, yolov8m, yolov8l
    'epochs': 10,
    'batch': 8,
    'imgsz': 640,
    'train_ratio': 0.85,
    'seed': 42,
}

# Image dimensions
IMG_WIDTH = 2592.0
IMG_HEIGHT = 2048.0
CLASS_ID = 0  # crater

print("✓ Configuration loaded")
print(f"  Model: {CONFIG['model']}")
print(f"  Epochs: {CONFIG['epochs']}")
print(f"  Batch: {CONFIG['batch']}")
print(f"  Image Size: {CONFIG['imgsz']}")

✓ Configuration loaded
  Model: yolov8n
  Epochs: 10
  Batch: 8
  Image Size: 640


## 4. Dataset Preparation

In [10]:
# def ellipse_to_yolo_bbox(cx, cy, semi_major, semi_minor, angle_deg):
#     """Convert ellipse to YOLO bbox (normalized)"""
#     angle = math.radians(angle_deg)
#     dx = math.sqrt((semi_major * math.cos(angle)) ** 2 + (semi_minor * math.sin(angle)) ** 2)
#     dy = math.sqrt((semi_major * math.sin(angle)) ** 2 + (semi_minor * math.cos(angle)) ** 2)
    
#     x1 = max(0.0, cx - dx)
#     x2 = min(IMG_WIDTH, cx + dx)
#     y1 = max(0.0, cy - dy)
#     y2 = min(IMG_HEIGHT, cy + dy)
    
#     w = x2 - x1
#     h = y2 - y1
#     if w <= 1e-6 or h <= 1e-6:
#         return None
    
#     cx_n = (x1 + x2) / (2.0 * IMG_WIDTH)
#     cy_n = (y1 + y2) / (2.0 * IMG_HEIGHT)
#     w_n = w / IMG_WIDTH
#     h_n = h / IMG_HEIGHT
    
#     if not (0.0 <= cx_n <= 1.0 and 0.0 <= cy_n <= 1.0):
#         return None
#     if w_n <= 0.0 or h_n <= 0.0:
#         return None
    
#     return (cx_n, cy_n, w_n, h_n)


# def read_annotations(gt_csv):
#     """Read crater annotations from CSV"""
#     annotations = {}
#     with gt_csv.open() as f:
#         reader = csv.DictReader(f)
#         for row in reader:
#             img_id = row["inputImage"]
#             cx = float(row["ellipseCenterX(px)"])
#             cy = float(row["ellipseCenterY(px)"])
#             semi_major = float(row["ellipseSemimajor(px)"])
#             semi_minor = float(row["ellipseSemiminor(px)"])
#             angle = float(row["ellipseRotation(deg)"])
#             annotations.setdefault(img_id, []).append((cx, cy, semi_major, semi_minor, angle))
#     return annotations


# def prepare_dataset(config):
#     """Prepare YOLO dataset"""
#     print("Preparing dataset...")
    
#     # Read annotations
#     annotations = read_annotations(config['gt_csv'])
#     print(f"  Loaded {len(annotations)} unique images")
    
#     # Split train/val
#     all_ids = list(annotations.keys())
#     random.Random(config['seed']).shuffle(all_ids)
#     split_idx = int(len(all_ids) * config['train_ratio'])
#     train_ids = sorted(all_ids[:split_idx])
#     val_ids = sorted(all_ids[split_idx:])
    
#     # Create directories
#     config['dataset_out'].mkdir(exist_ok=True)
    
#     # Process each split
#     for split_name, img_ids in [("train", train_ids), ("val", val_ids)]:
#         img_count = 0
#         for img_id in img_ids:
#             src_img = config['images_root'] / f"{img_id}.png"
#             if not src_img.exists():
#                 continue
            
#             # Copy image
#             dst_img = config['dataset_out'] / "images" / split_name / f"{img_id}.png"
#             dst_img.parent.mkdir(parents=True, exist_ok=True)
#             shutil.copy2(src_img, dst_img)
            
#             # Create label file
#             dst_lbl = config['dataset_out'] / "labels" / split_name / f"{img_id}.txt"
#             dst_lbl.parent.mkdir(parents=True, exist_ok=True)
            
#             bboxes = []
#             for cx, cy, semi_major, semi_minor, angle in annotations.get(img_id, []):
#                 bbox = ellipse_to_yolo_bbox(cx, cy, semi_major, semi_minor, angle)
#                 if bbox:
#                     bboxes.append(bbox)
            
#             with dst_lbl.open('w') as f:
#                 for cx, cy, w, h in bboxes:
#                     f.write(f"{CLASS_ID} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
            
#             img_count += 1
        
#         print(f"  {split_name.upper()}: {img_count} images")
    
#     print("✓ Dataset prepared")


# # Prepare dataset
# prepare_dataset(CONFIG)

In [11]:
def ellipse_to_yolo_bbox(cx, cy, semi_major, semi_minor, angle_deg):
    """Convert ellipse to YOLO bbox (normalized)"""
    angle = math.radians(angle_deg)
    dx = math.sqrt((semi_major * math.cos(angle)) ** 2 + (semi_minor * math.sin(angle)) ** 2)
    dy = math.sqrt((semi_major * math.sin(angle)) ** 2 + (semi_minor * math.cos(angle)) ** 2)
    
    x1 = max(0.0, cx - dx)
    x2 = min(IMG_WIDTH, cx + dx)
    y1 = max(0.0, cy - dy)
    y2 = min(IMG_HEIGHT, cy + dy)
    
    w = x2 - x1
    h = y2 - y1
    if w <= 1e-6 or h <= 1e-6:
        return None
    
    cx_n = (x1 + x2) / (2.0 * IMG_WIDTH)
    cy_n = (y1 + y2) / (2.0 * IMG_HEIGHT)
    w_n = w / IMG_WIDTH
    h_n = h / IMG_HEIGHT
    
    if not (0.0 <= cx_n <= 1.0 and 0.0 <= cy_n <= 1.0):
        return None
    if w_n <= 0.0 or h_n <= 0.0:
        return None
    
    return (cx_n, cy_n, w_n, h_n)


def read_annotations(gt_csv):
    """Read crater annotations from CSV"""
    annotations = {}
    with gt_csv.open() as f:
        reader = csv.DictReader(f)
        for row in reader:
            img_id = row["inputImage"]
            cx = float(row["ellipseCenterX(px)"])
            cy = float(row["ellipseCenterY(px)"])
            semi_major = float(row["ellipseSemimajor(px)"])
            semi_minor = float(row["ellipseSemiminor(px)"])
            angle = float(row["ellipseRotation(deg)"])
            annotations.setdefault(img_id, []).append((cx, cy, semi_major, semi_minor, angle))
    return annotations


def prepare_dataset(config):
    """Prepare YOLO dataset"""
    print("Preparing dataset...")
    
    # Read annotations
    annotations = read_annotations(config['gt_csv'])
    print(f"  Loaded {len(annotations)} unique images")
    
    # Split train/val
    all_ids = list(annotations.keys())
    random.Random(config['seed']).shuffle(all_ids)
    split_idx = int(len(all_ids) * config['train_ratio'])
    train_ids = sorted(all_ids[:split_idx])
    val_ids = sorted(all_ids[split_idx:])

    # --- Reduce dataset size ---
    train_ids = train_ids[:1000]
    val_ids = val_ids[:50]
    print(f"  Reduced dataset to: Train={len(train_ids)}, Val={len(val_ids)}")
    # ---------------------------
    
    # Create directories
    config['dataset_out'].mkdir(exist_ok=True)
    
    # Process each split
    for split_name, img_ids in [("train", train_ids), ("val", val_ids)]:
        img_count = 0
        for img_id in img_ids:
            src_img = config['images_root'] / f"{img_id}.png"
            if not src_img.exists():
                continue
            
            # Copy image
            dst_img = config['dataset_out'] / "images" / split_name / f"{img_id}.png"
            dst_img.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src_img, dst_img)
            
            # Create label file
            dst_lbl = config['dataset_out'] / "labels" / split_name / f"{img_id}.txt"
            dst_lbl.parent.mkdir(parents=True, exist_ok=True)
            
            bboxes = []
            for cx, cy, semi_major, semi_minor, angle in annotations.get(img_id, []):
                bbox = ellipse_to_yolo_bbox(cx, cy, semi_major, semi_minor, angle)
                if bbox:
                    bboxes.append(bbox)
            
            with dst_lbl.open('w') as f:
                for cx, cy, w, h in bboxes:
                    f.write(f"{CLASS_ID} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
            
            img_count += 1
        
        print(f"  {split_name.upper()}: {img_count} images")
    
    print("✓ Dataset prepared")


# Prepare dataset
prepare_dataset(CONFIG)

Preparing dataset...
  Loaded 4150 unique images
  Reduced dataset to: Train=1000, Val=50
  TRAIN: 1000 images
  VAL: 50 images
✓ Dataset prepared


In [12]:
bhgbvh

NameError: name 'bhgbvh' is not defined

## 5. Create YOLO Config

In [ ]:
# Create crater.yaml
yaml_content = f"""path: {CONFIG['dataset_out'].resolve()}
train: images/train
val: images/val
nc: 1
names:
  0: crater
"""

yaml_path = Path('./crater.yaml')
yaml_path.write_text(yaml_content)
print(f"✓ Created {yaml_path}")

✓ Created crater.yaml


## 6. Train Model

In [ ]:
# Get device
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load model
print(f"Loading {CONFIG['model']}...")
model = YOLO(f"{CONFIG['model']}.pt")

# Train
print("\nStarting training...")
results = model.train(
    data='crater.yaml',
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    device=device,
    project=str(CONFIG['runs_dir']),
    patience=10,
    save=True,
    plots=True,
    verbose=True,
)

print("✓ Training completed")

Using device: cpu
Loading yolov8n...

Starting training...
Ultralytics 8.3.252  Python-3.12.0 torch-2.9.1+cpu CPU (11th Gen Intel Core i3-1115G4 @ 3.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=crater.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

## 7. Validate Model

In [ ]:
# Get best weights path
runs_detect = CONFIG['runs_dir'] 
latest_run = sorted(runs_detect.glob('*'))[-1]
best_weights = latest_run / 'weights' / 'best.pt'

print(f"Best weights: {best_weights}")

# Load best model
best_model = YOLO(str(best_weights))

# Validate
print("\nValidating...")
metrics = best_model.val(data='crater.yaml', imgsz=CONFIG['imgsz'], device=device, split='val')

print("\n✓ Validation Results:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

Best weights: runs\train3\weights\best.pt

Validating...
Ultralytics 8.3.252  Python-3.12.0 torch-2.9.1+cpu CPU (11th Gen Intel Core i3-1115G4 @ 3.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 708.0296.1 MB/s, size: 2575.5 KB)
val: Scanning D:\datashare\yolo\dataset\labels\val\altitude01\longitude02.cache... 5 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5/5  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 1.0s/it 1.0s
                   all          5         99      0.044      0.667      0.307       0.22
Speed: 4.8ms preprocess, 116.4ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to D:\datashare\yolo\runs\detect\val

✓ Validation Results:
  mAP50: 0.3066
  mAP: 0.2196
  Precision: 0.0440
  Recall: 0.6667


## 8. Test Inference

In [ ]:
# Get test images
test_dir = Path('../train/train/altitude08/longitude15')
test_images = sorted(test_dir.glob('orientation*.png'))[:3]

print(f"Testing on {len(test_images)} images...")

# Run inference
results = best_model.predict(
    source=test_images,
    conf=0.25,
    imgsz=CONFIG['imgsz'],
    device=device,
    save=True,
    project='./predictions',
    name='test',
)

# Summary
total_detections = sum(len(r.boxes) for r in results)
print(f"\n✓ Inference Results:")
print(f"  Total detections: {total_detections}")
print(f"  Avg detections per image: {total_detections / len(test_images):.1f}")
print(f"  Results saved to: ./predictions/test")

Testing on 3 images...

0: 512x640 (no detections), 184.1ms
1: 512x640 (no detections), 184.1ms
2: 512x640 (no detections), 184.1ms
Speed: 3.5ms preprocess, 184.1ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)
Results saved to D:\datashare\yolo\predictions\test

✓ Inference Results:
  Total detections: 0
  Avg detections per image: 0.0
  Results saved to: ./predictions/test


## 9. Export Model

In [ ]:
# # Export to ONNX
# print("Exporting to ONNX...")
# exported_path = best_model.export(
#     format='onnx',
#     imgsz=CONFIG['imgsz'],
#     device=device,
# )

# print(f"✓ Model exported to: {exported_path}")
# print(f"\n✓ Training complete!")
# print(f"  Best weights: {best_weights}")
# print(f"  Exported: {exported_path}")

In [ ]:
# Function to draw Yolo bboxes
def draw_yolo_labels(image_path, label_path):
    # Read image
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Error: Could not read image {image_path}")
        return
    
    h_img, w_img = img.shape[:2]
    
    # Read labels
    if not Path(label_path).exists():
        print(f"Error: Label file not found {label_path}")
        return

    with open(label_path, 'r') as f:
        lines = f.readlines()
        
    print(f"Drawing {len(lines)} boxes on {Path(image_path).name}...")
        
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 5:
            # YOLO format: class x_center y_center width height (normalized)
            cls, cx_n, cy_n, w_n, h_n = map(float, parts[:5])
            
            # Convert to pixel coordinates
            w_px = w_n * w_img
            h_px = h_n * h_img
            cx_px = cx_n * w_img
            cy_px = cy_n * h_img
            
            x1 = int(cx_px - w_px / 2)
            y1 = int(cy_px - h_px / 2)
            x2 = int(cx_px + w_px / 2)
            y2 = int(cy_px + h_px / 2)
            
            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, str(int(cls)), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save output to view it easily (or display it if using appropriate display tools)
    out_path = Path('./drawn_labels_check.png')
    cv2.imwrite(str(out_path), img)
    print(f"Saved visualization to {out_path.resolve()}")

# Define paths (Adjusting based on your request and likely relative paths in notebook context)
# Assuming relative path from notebook root based on previous cells
target_lbl = Path('dataset/labels/train/altitude01_longitude02_orientation01_light01.txt') # Note: previous cell flattened hierarchy in dataset/labels/train/
target_img = Path('dataset/images/train/altitude01_longitude02_orientation01_light01.png')

# If the user specifically provided absolute paths or a different structure in the query, we try to use those
# The user prompt has specific paths with subfolders: altitude01/longitude02... 
# But the previous code (CELL 9) flattens the dataset structure: dataset/images/train/{img_id}.png
# Let's try to construct the path based on the likely img_id from the user prompt filename.
img_id = "altitude01_longitude02_orientation01_light01" # Usually constructed from flattened path
# If the previous cells didn't flatten the structure, we would use grid.

# Let's construct paths based on the CONFIG['dataset_out'] variable available in context
# and the user's specific request structure.
user_img_path = Path(r"D:\datashare\yolo\dataset\images\train\altitude01\longitude02\orientation01_light01.png")
user_lbl_path = Path(r"D:\datashare\yolo\dataset\labels\train\altitude01\longitude02\orientation01_light01.txt")

# Check if file exists, if not, try the flattened path standard in cell 9
if not user_img_path.exists():
    img_id_guess = "altitude01_longitude02_orientation01_light01"
    user_img_path = CONFIG['dataset_out'] / 'images' / 'train' / f"{img_id_guess}.png"
    user_lbl_path = CONFIG['dataset_out'] / 'labels' / 'train' / f"{img_id_guess}.txt"

draw_yolo_labels(user_img_path, user_lbl_path)

Drawing 26 boxes on orientation01_light01.png...
Saved visualization to D:\datashare\yolo\drawn_labels_check.png


In [ ]:
# Function to draw YOLO bboxes + original ellipses + ellipse AABB
def _infer_annotation_key_from_image_path(image_path):
    p = Path(image_path)
    parts = list(p.parts)
    # Try nested dir form: .../altitudeXX/longitudeYY/orientationZZ_lightPP.png
    for i, s in enumerate(parts):
        if s.startswith('altitude') and i + 2 < len(parts):
            alt = parts[i]
            lon = parts[i + 1]
            ori = parts[i + 2]
            ori = Path(ori).stem  # remove .png if present
            if lon.startswith('longitude') and ori.startswith('orientation'):
                return f"{alt}/{lon}/{ori}"
    # Fallback: flattened stem altitudeXX_longitudeYY_orientationZZ_lightPP
    stem = Path(image_path).stem
    toks = stem.split('_')
    if len(toks) >= 4 and toks[0].startswith('altitude') and toks[1].startswith('longitude'):
        return f"{toks[0]}/{toks[1]}/{toks[2]}_{toks[3]}"
    return None

def _normalize_img_id_for_annotations(img_id):
    # Accept either "altitude..../longitude..../orientation.._light.." or flattened form
    if img_id and '/' in img_id:
        return img_id
    if img_id:
        toks = Path(img_id).stem.split('_')
        if len(toks) >= 4:
            return f"{toks[0]}/{toks[1]}/{toks[2]}_{toks[3]}"
    return None

def draw_yolo_labels_with_ellipses(image_path, label_path, annotations_dict=None, img_id=None, out_path=None):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Error: Could not read image {image_path}")
        return

    h_img, w_img = img.shape[:2]

    # 1) Draw YOLO Bounding Boxes (Green)
    if Path(label_path).exists():
        with open(label_path, 'r') as f:
            lines = f.readlines()
        print(f"Drawing {len(lines)} YOLO boxes on {Path(image_path).name}...")
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls, cx_n, cy_n, w_n, h_n = map(float, parts[:5])
                w_px = w_n * w_img
                h_px = h_n * h_img
                cx_px = cx_n * w_img
                cy_px = cy_n * h_img
                x1 = int(cx_px - w_px / 2)
                y1 = int(cy_px - h_px / 2)
                x2 = int(cx_px + w_px / 2)
                y2 = int(cy_px + h_px / 2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, str(int(cls)), (x1, max(0, y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        print(f"Warning: Label file not found {label_path}")

    # 2) Draw original Ellipses (Blue) and their axis-aligned bbox (Yellow)
    ann_key = None
    if annotations_dict is not None:
        ann_key = _normalize_img_id_for_annotations(img_id) if img_id else None
        if not ann_key:
            ann_key = _infer_annotation_key_from_image_path(image_path)

        if ann_key and ann_key in annotations_dict:
            ellipses = annotations_dict[ann_key]
            print(f"Drawing {len(ellipses)} ellipses for key '{ann_key}'...")
            for cx, cy, semi_major, semi_minor, angle in ellipses:
                # Ellipse (Blue)
                center = (int(round(cx)), int(round(cy)))
                axes = (int(round(semi_major)), int(round(semi_minor)))
                cv2.ellipse(img, center, axes, angle, 0, 360, (255, 0, 0), 2)

                # Ellipse axis-aligned bounding box (Yellow)
                ang = math.radians(angle)
                dx = math.sqrt((semi_major * math.cos(ang)) ** 2 + (semi_minor * math.sin(ang)) ** 2)
                dy = math.sqrt((semi_major * math.sin(ang)) ** 2 + (semi_minor * math.cos(ang)) ** 2)
                x1 = int(max(0, min(w_img - 1, cx - dx)))
                y1 = int(max(0, min(h_img - 1, cy - dy)))
                x2 = int(max(0, min(w_img - 1, cx + dx)))
                y2 = int(max(0, min(h_img - 1, cy + dy)))
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        else:
            print(f"Warning: No annotations found for key '{ann_key}'")
    else:
        print("Warning: annotations_dict is None; skipping ellipse drawing")

    # Save output
    if out_path is None:
        out_path = Path('./drawn_labels_ellipses_check.png')
    cv2.imwrite(str(out_path), img)
    print(f"Saved visualization to {Path(out_path).resolve()}")

# --- Setup and run on one sample ---
annotations_all = read_annotations(CONFIG['gt_csv'])

# Try a known sample; auto-fallback to nested structure if needed
img_id_guess = "altitude01_longitude02_orientation01_light01"
test_img_path = CONFIG['dataset_out'] / 'images' / 'train' / f"{img_id_guess}.png"
test_lbl_path = CONFIG['dataset_out'] / 'labels' / 'train' / f"{img_id_guess}.txt"

# If flattened file doesn't exist, try nested folder structure
if not test_img_path.exists():
    nested_img = CONFIG['dataset_out'] / 'images' / 'train' / 'altitude01' / 'longitude02' / 'orientation01_light01.png'
    nested_lbl = CONFIG['dataset_out'] / 'labels' / 'train' / 'altitude01' / 'longitude02' / 'orientation01_light01.txt'
    if nested_img.exists():
        test_img_path, test_lbl_path = nested_img, nested_lbl

# Output path per image name
out_vis = Path('./drawn_labels_ellipses_check.png')
draw_yolo_labels_with_ellipses(test_img_path, test_lbl_path, annotations_all, img_id_guess, out_vis)

Drawing 26 YOLO boxes on orientation01_light01.png...
Drawing 26 ellipses for key 'altitude01/longitude02/orientation01_light01'...
Saved visualization to D:\datashare\yolo\drawn_labels_ellipses_check.png
